# IMPORTS

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

# INICIAR EL DRIVER Y LOGGEANDO LINKEDIN

In [2]:
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    #options.add_argument("headless")
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    #options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    s = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=s, options=options)


In [3]:
def linkedin_login(driver):
    soup = BeautifulSoup(page_source, "html.parser")
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    login_button = soup.find("button", {"class": "button[btn-md btn-primary flex-shrink-0 cursor-pointersign-in-form__submit-btn--full-width"})
    login_button.click()
    


# DA POSITIONS

In [4]:
def data_analyst():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument(f'user-agent={user_agent}')
    #options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    s = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, options=options)

    linkedin_login(driver)

    titles = []
    companies = []
    locations = []
    modalities = []
    descriptions = []
    type_and_levels = []

    for i in tqdm(range(0, 2)):
        driver.get(f'https://www.linkedin.com/jobs/search/?currentJobId=3521795653&geoId=107025191&keywords=data%20analyst&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true={i}')
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")
        all_listings = []
        keep_scrolling = True

        while keep_scrolling:
            listings = driver.find_elements(By.CSS_SELECTOR, ".job-card-list__title")
            if set(listings) == set(all_listings):
                keep_scrolling = False
            else:
                new_listings = list(set(listings) - set(all_listings))
                for listing in new_listings:
                    all_listings.append(listing)
                    try:
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth'});", listing)
                        listing.click()
                        time.sleep(random.randint(2, 5))
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        try:
                            title = soup.find("h2", {"class": "t-24 t-bold jobs-unified-top-card__job-title"}).get_text(strip=True)

                            if 'data' in title.lower() or 'datos' in title.lower():
                                titles.append(title)
                                try:
                                    company = soup.find("span", {"class": "jobs-unified-top-card__company-name"}).get_text(strip=True)
                                    companies.append(company)
                                except AttributeError:
                                    companies.append('Unknown')
                                try:
                                    location = soup.find("span", {"class": "jobs-unified-top-card__bullet"}).get_text(strip=True)
                                    locations.append(location)
                                except AttributeError:
                                    locations.append('Unknown')
                                try:
                                    modality = soup.find("span", {"class": "jobs-unified-top-card__workplace-type"}).get_text(strip=True)
                                    modalities.append(modality)
                                except AttributeError:
                                    modalities.append('Unknown')
                                try:
                                    description = soup.find("div", {"class": "jobs-description__content jobs-description-content"}).get_text(strip=True)
                                    descriptions.append(description)
                                except AttributeError:
                                    descriptions.append('Unknown')
                                try:
                                    span_element = soup.find('span')
                                    type_and_level = span_element.get_text(strip=True)
                                    type_and_levels.append(type_and_level)
                                except AttributeError:
                                    type_and_levels.append('Unknown')

                        except AttributeError:
                            pass
                    except:
                        pass

    driver.quit()

    df = pd.DataFrame({'title': titles, 'company': companies, 'location': locations,
                       'modality': modalities, 'description': descriptions, 'type_and_level': type_and_levels})
    return df

analyst = data_analyst()

NameError: name 'page_source' is not defined

In [ ]:
analyst = data_analyst()
